In [2]:
import shutil
import glob
import json
import os
import hashlib

shutil.rmtree("./data", ignore_errors=True)
os.mkdir("./data")

data_json: list[tuple[str, tuple[str, str], tuple[str, str, str, str, str, str]]] = []

categories = [("Seen Source and Target", "seen"), ("Unseen Source and Target", "unseen")]

for dir_name, key_name in categories:
    category_name = key_name[0].upper() + key_name[1:]
    for src_tgt in glob.glob(f"./__src__/{dir_name}/*"):
        src_lang, tgt_lang = src_tgt.split(os.path.sep)[-1].split("_")[0].split("2")
        wavs = ["" for _ in range(6)]

        for wav_path in glob.glob(f"{src_tgt}/*.wav"):
            if os.path.basename(wav_path).startswith("s"):
                wavs[0] = wav_path
            elif os.path.basename(wav_path).startswith("t"):
                wavs[1] = wav_path
            elif os.path.basename(wav_path).startswith("ac"):
                wavs[2] = wav_path
            elif os.path.basename(wav_path).startswith("vq"):
                wavs[3] = "pass" # wav_path
            elif os.path.basename(wav_path).startswith("au"):
                wavs[4] = wav_path
            elif os.path.basename(wav_path).startswith("ag"):
                wavs[5] = "pass" # wav_path

        md5_wavs = []
        for wavs_idx, wav_path in enumerate(wavs):
            assert wav_path != "", f"{src_tgt} {wavs_idx}"
            if wav_path == "pass":
                md5_wavs.append("")
                continue

            # move the file to "./data/{md5_hash}.wav"
            md5_hash = hashlib.md5(open(wav_path, "rb").read()).hexdigest()
            shutil.copy(wav_path, f"./data/{md5_hash}.wav")

            md5_wavs.append(md5_hash)

        data_json.append(
            (
                category_name,
                (src_lang, tgt_lang),
                tuple(md5_wavs),
            )
        )

with open("./data.json", "w") as f:
    json.dump(data_json, f)